<a href="https://colab.research.google.com/github/pipuf/ml_dev_cert/blob/main/3_3_2_THEORY_SQL_with_Python.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Databases

## Introduction

A huge amount of data lives in databases, so it's essential that you know how to access them. Sometimes you can ask someone to download a snapshot into a .csv for you, but this gets painful quickly: every time you need to make a change you'll have to communicate with another human. You want to be able to reach into the database directly to get the data you need, when you need it.

In this chapter, you'll first learn the basics of Python packages that can interact with SQL databases.

## Connecting to Databases

To connect to a database from Python, there are a number of options, but it does depend on what *kind* of database you are connecting to. The steps (eg connect to the database, upload data, run SQL series) are similar though, so we won't lose much by just choosing one example.

SQLite is a small, fast, self-contained, SQL database engine and the most used database engine in the world. A lot of the data on your computer and mobile phone will in fact be in SQLite databases.

We will always need two steps though, regardless of what kind of SQL database we are connecting to:

- You'll always use a database interface that provides a connection to the database, for example Python's built-in **sqlite** package.
- You'll also use a package that pushes and/or pulls data to/from the database, for example **pandas**.


## Working with Databases Directly

Let's connect to a small SQLite database called the [Chinook database](https://github.com/lerocha/chinook-database), which contains information about the artists, songs, and albums from a music shop, as well as information on the shop's employees, customers, and the customers purchases. This information is contained in eleven tables. The figure below shows the data schema:

For starters, let's use Python's built in **sqlite3** engine to connect to the database and execute a very simple SQL query to select the first ten entries from the 'Artists' table:

In [ ]:
import sqlite3

import pandas as pd


In [ ]:
# We need to first download the .sqlite database file
!gdown "1Pedy3vxwYWFOJjADGYBRN2IA95glpIfF"

Downloading...
From: https://drive.google.com/uc?id=1Pedy3vxwYWFOJjADGYBRN2IA95glpIfF
To: /content/Chinook_Sqlite.sqlite
100% 1.07M/1.07M [00:00<00:00, 92.0MB/s]


In [ ]:
# We can establish a connection to the database with this command
con = sqlite3.connect("Chinook_Sqlite.sqlite")


In [ ]:
# List all the tables
# First we must type the SQL command and execute it with execute()
# In SQLite, sqlite_master is a special system table that stores metadata about
# the database schema. It contains information about all the tables, indexes,
# views, and triggers in the database.

cursor = con.execute(
  """
  SELECT name
  FROM sqlite_master
  WHERE type='table';
  """
)

# Then we can retreive the results using fetchall()
rows = cursor.fetchall()
rows

[('Album',),
 ('Artist',),
 ('Customer',),
 ('Employee',),
 ('Genre',),
 ('Invoice',),
 ('InvoiceLine',),
 ('MediaType',),
 ('Playlist',),
 ('PlaylistTrack',),
 ('Track',)]

In [ ]:
# Let's see another example
# Listing only 10 artists
cursor = con.execute(
  """
  SELECT *
  FROM Artist
  LIMIT 50;
  """
)

rows = cursor.fetchall()
rows

[(1, 'AC/DC'),
 (2, 'Accept'),
 (3, 'Aerosmith'),
 (4, 'Alanis Morissette'),
 (5, 'Alice In Chains'),
 (6, 'Antônio Carlos Jobim'),
 (7, 'Apocalyptica'),
 (8, 'Audioslave'),
 (9, 'BackBeat'),
 (10, 'Billy Cobham'),
 (11, 'Black Label Society'),
 (12, 'Black Sabbath'),
 (13, 'Body Count'),
 (14, 'Bruce Dickinson'),
 (15, 'Buddy Guy'),
 (16, 'Caetano Veloso'),
 (17, 'Chico Buarque'),
 (18, 'Chico Science & Nação Zumbi'),
 (19, 'Cidade Negra'),
 (20, 'Cláudio Zoli'),
 (21, 'Various Artists'),
 (22, 'Led Zeppelin'),
 (23, 'Frank Zappa & Captain Beefheart'),
 (24, 'Marcos Valle'),
 (25, 'Milton Nascimento & Bebeto'),
 (26, 'Azymuth'),
 (27, 'Gilberto Gil'),
 (28, 'João Gilberto'),
 (29, 'Bebel Gilberto'),
 (30, 'Jorge Vercilo'),
 (31, 'Baby Consuelo'),
 (32, 'Ney Matogrosso'),
 (33, 'Luiz Melodia'),
 (34, 'Nando Reis'),
 (35, 'Pedro Luís & A Parede'),
 (36, 'O Rappa'),
 (37, 'Ed Motta'),
 (38, 'Banda Black Rio'),
 (39, 'Fernanda Porto'),
 (40, 'Os Cariocas'),
 (41, 'Elis Regina'),
 (42, 'Mi

In [ ]:
type(rows[0])

tuple

Note that the output here is in the form a Python object called a tuple. If we wanted to put this into a **pandas** dataframe, we can just pass it straight in:

In [ ]:
import pandas as pd

pd.DataFrame(rows).head()

,0,1
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


Another useful hint is that if you're not sure what the column names are, you can obtain them from:

In [ ]:
[i[0] for i in cursor.description]

['ArtistId', 'Name']

Putting all togheter...

In [ ]:
df = pd.DataFrame(rows, columns=[i[0] for i in cursor.description])
df.head()

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains


In [ ]:
df.set_index("ArtistId").head()

,Name
ArtistId,
1,AC/DC
2,Accept
3,Aerosmith
4,Alanis Morissette
5,Alice In Chains


### Creating a Database

Often, you'll want to create a SQL database to later (efficiently) access cuts of data. Let's create a test database directly using the **sqlite** package. This process involves a `CREATE TABLE` statement, then the name of the table followed by the names of the columns and their data types.

In [ ]:
create_query = """CREATE TABLE test (country VARCHAR(20), gdp REAL, health INTEGER);"""
con_new = sqlite3.connect("test3_database.sqlite")
con_new.execute(create_query)
con_new.commit()


If this runs, then you've created a test database! (You can check the data directory on your own computer to see if it worked, but you'll get an error if the data directory doesn't already exist.)

Let's now fill the database with some values:

In [ ]:
test_data = [("MX", 1, 3), ("Col", 0.6, 2), ("Argentina", 0.8, 1)]

con_new.executemany("INSERT INTO test VALUES(?, ?, ?)", test_data)
con_new.commit()

Finally, let's check if this has worked:

In [ ]:
con_new.execute("SELECT * FROM test").fetchall()

[('US', 1.0, 3),
 ('UK', 0.6, 2),
 ('France', 0.8, 1),
 ('US', 1.0, 3),
 ('UK', 0.6, 2),
 ('France', 0.8, 1),
 ('US', 1.0, 3),
 ('UK', 0.6, 2),
 ('France', 0.8, 1),
 ('MX', 1.0, 3),
 ('Col', 0.6, 2),
 ('Argentina', 0.8, 1),
 ('MX', 1.0, 3),
 ('Col', 0.6, 2),
 ('Argentina', 0.8, 1)]

## SQL with **pandas**

**Pandas** is well-equipped for working with SQL. We can simply push the query we just created straight through using its `read_sql` function—but bear in mind we need to pass in the connection we created to the database too:

In [ ]:
con = sqlite3.connect("Chinook_Sqlite.sqlite")

In [ ]:
sql_top10_artists = """
  SELECT *
  FROM Artist
  LIMIT 100;
"""

df_top10_artists = pd.read_sql(sql_top10_artists, con)
df_top10_artists

,ArtistId,Name
0,1,AC/DC
1,2,Accept
2,3,Aerosmith
3,4,Alanis Morissette
4,5,Alice In Chains
...,...,...
95,96,Jota Quest
96,97,João Suplicy
97,98,Judas Priest
98,99,Legião Urbana


One nice feature of this is that the column names in SQL get passed straight to the column names in our dataframe.

### Creating a SQL Table from Pandas Dataframe

Now I will use the table above to show how you can also create your own tables in the SQLite Database using pandas:


In [ ]:
type(df_top10_artists)

pandas.core.frame.DataFrame

In [ ]:
df_top10_artists.to_sql('Top100_Artist', con)


100

In [ ]:
# List all the tables, Top10_Artist should be included now
sql_list_tables = """
  SELECT name
  FROM sqlite_master
  WHERE type='table';
  """

pd.read_sql(sql_list_tables, con)


,name
0,Album
1,Artist
2,Customer
3,Employee
4,Genre
5,Invoice
6,InvoiceLine
7,MediaType
8,Playlist
9,PlaylistTrack


### Examples of SQL queries with Pandas

In [ ]:
sql_query = """
-- SELECT example: Select InvoiceId, BillingAddress and BillingCity columns from Invoice table. (This is a comment and won’t be executed)
SELECT InvoiceId, BillingAddress, BillingCity
FROM Invoice;
"""

pd.read_sql(sql_query, con)

,InvoiceId,BillingAddress,BillingCity
0,1,Theodor-Heuss-Straße 34,Stuttgart
1,2,Ullevålsveien 14,Oslo
2,3,Grétrystraat 63,Brussels
3,4,8210 111 ST NW,Edmonton
4,5,69 Salem Street,Boston
...,...,...,...
407,408,319 N. Frances Street,Madison
408,409,796 Dundas Street West,Toronto
409,410,"Rua dos Campeões Europeus de Viena, 4350",Porto
410,411,Porthaninkatu 9,Helsinki


In [ ]:
sql_query = """
-- WHERE example: Select all the invoices billed from Argentina in which the spent value is over $5
SELECT *
FROM Invoice
WHERE Invoice.BillingCountry = 'Argentina' AND Invoice.Total > 5;
"""

pd.read_sql(sql_query, con)

,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,164,56,2010-12-17 00:00:00,307 Macacha Güemes,Buenos Aires,None,Argentina,1106,5.94
1,348,56,2013-03-10 00:00:00,307 Macacha Güemes,Buenos Aires,None,Argentina,1106,13.86
2,403,56,2013-11-08 00:00:00,307 Macacha Güemes,Buenos Aires,None,Argentina,1106,8.91


In [ ]:
df = pd.read_csv("data.csv")
df.to_sql("test", con)

In [ ]:
sql_query = """
-- More complex example with alias
SELECT *
FROM Customer as c, Invoice AS i
WHERE c.FirstName = 'Mark' AND c.CustomerId = i.CustomerId;
"""

pd.read_sql(sql_query, con)

,CustomerId,FirstName,LastName,Company,Address,City,State,Country,PostalCode,Phone,...,SupportRepId,InvoiceId,CustomerId,InvoiceDate,BillingAddress,BillingCity,BillingState,BillingCountry,BillingPostalCode,Total
0,14,Mark,Philips,Telus,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,+1 (780) 434-4554,...,5,4,14,2009-01-06 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,8.91
1,14,Mark,Philips,Telus,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,+1 (780) 434-4554,...,5,133,14,2010-08-13 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,1.98
2,14,Mark,Philips,Telus,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,+1 (780) 434-4554,...,5,156,14,2010-11-15 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,3.96
3,14,Mark,Philips,Telus,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,+1 (780) 434-4554,...,5,178,14,2011-02-17 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,5.94
4,14,Mark,Philips,Telus,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,+1 (780) 434-4554,...,5,230,14,2011-10-08 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,0.99
5,14,Mark,Philips,Telus,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,+1 (780) 434-4554,...,5,351,14,2013-03-31 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,1.98
6,14,Mark,Philips,Telus,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,+1 (780) 434-4554,...,5,362,14,2013-05-11 00:00:00,8210 111 ST NW,Edmonton,AB,Canada,T6G 2C7,13.86
7,55,Mark,Taylor,None,421 Bourke Street,Sidney,NSW,Australia,2010,+61 (02) 9332 3633,...,4,21,55,2009-04-04 00:00:00,421 Bourke Street,Sidney,NSW,Australia,2010,1.98
8,55,Mark,Taylor,None,421 Bourke Street,Sidney,NSW,Australia,2010,+61 (02) 9332 3633,...,4,44,55,2009-07-07 00:00:00,421 Bourke Street,Sidney,NSW,Australia,2010,3.96
9,55,Mark,Taylor,None,421 Bourke Street,Sidney,NSW,Australia,2010,+61 (02) 9332 3633,...,4,66,55,2009-10-09 00:00:00,421 Bourke Street,Sidney,NSW,Australia,2010,5.94


In [ ]:
sql_query = """
-- GROUP BY example: Number of sales and average amount spent by Country during 2010
SELECT COUNT(*) as Sales_by_Country, AVG(i.Total) as Average_spent, i.BillingCountry as Country
FROM Invoice AS i
WHERE i.InvoiceDate BETWEEN '2010-01-01 00:00:00' and '2010-31-12 00:00:00'
GROUP BY i.BillingCountry
ORDER BY Sales_by_Country DESC;
"""

pd.read_sql(sql_query, con)


,Sales_by_Country,Average_spent,Country
0,18,5.721111,USA
1,12,6.355000,Canada
2,8,4.950000,France
3,8,5.200000,Brazil
4,5,6.138000,United Kingdom
5,4,6.435000,Germany
6,3,2.310000,Portugal
7,3,3.630000,Italy
8,3,5.943333,India
9,3,10.916667,Hungary


In [ ]:
sql_query = """
-- GROUP BY example: Number of sales and average amount spent by Country, only for those who had more than 10 sales
SELECT COUNT(*) as Sales_by_Country, AVG(i.Total) as Average_spent, i.BillingCountry as Country
FROM Invoice AS i
GROUP BY i.BillingCountry
HAVING Sales_by_Country > 10
ORDER BY Sales_by_Country DESC;
"""

pd.read_sql(sql_query, con)

,Sales_by_Country,Average_spent,Country
0,91,5.747912,USA
1,56,5.427857,Canada
2,35,5.574286,France
3,35,5.431429,Brazil
4,28,5.588571,Germany
5,21,5.374286,United Kingdom
6,14,5.517143,Portugal
7,14,6.445714,Czech Republic
8,13,5.789231,India


In [ ]:
sql_query = """
-- CASE + GRUOP BY example: Count Tracks by Quality
SELECT case_result.AudioQuality, COUNT(*) FROM (
    SELECT *,
    (CASE
    	WHEN Track.Bytes < 2000000 THEN 'Poor-Quality'
        WHEN Track.Bytes >= 2000000 AND Track.Bytes <= 8000000 THEN 'Mid-Quality'
        WHEN Track.Bytes > 8000000 THEN 'High-Quality'
    END) AS AudioQuality
    FROM Track
    ORDER BY Track.Bytes ASC
) AS case_result
GROUP BY case_result.AudioQuality;
"""

pd.read_sql(sql_query, con)

,AudioQuality,COUNT(*)
0,High-Quality,1814
1,Mid-Quality,1654
2,Poor-Quality,35


In [ ]:
sql_query = """
-- INNER JOIN: Employee asigned to each Customer
SELECT c.CustomerId, e.EmployeeId, e.FirstName, e.LastName, e.Title
FROM Customer AS c
INNER JOIN Employee as e
ON c.SupportRepId = e.EmployeeId;
"""

pd.read_sql(sql_query, con)

,CustomerId,EmployeeId,FirstName,LastName,Title
0,1,3,Jane,Peacock,Sales Support Agent
1,3,3,Jane,Peacock,Sales Support Agent
2,12,3,Jane,Peacock,Sales Support Agent
3,15,3,Jane,Peacock,Sales Support Agent
4,18,3,Jane,Peacock,Sales Support Agent
5,19,3,Jane,Peacock,Sales Support Agent
6,24,3,Jane,Peacock,Sales Support Agent
7,29,3,Jane,Peacock,Sales Support Agent
8,30,3,Jane,Peacock,Sales Support Agent
9,33,3,Jane,Peacock,Sales Support Agent
